In [ ]:
#pip install kagglehub

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
datamunge_sign_language_mnist_path = kagglehub.dataset_download('datamunge/sign-language-mnist')

print('Data source import complete.')

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # data visualization
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):                                        # SOLO PARA KAGGLE
for dirname, _, filenames in os.walk(datamunge_sign_language_mnist_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

n_jobs = -1 # number of jobs to run in parallel
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Cargo los datos y los separo en desarrollo y evaluación final

In [ ]:
data_eval = pd.read_csv(r'C:\Users\aaron\.cache\kagglehub\datasets\datamunge\sign-language-mnist\versions\1\sign_mnist_test\sign_mnist_test.csv')
data_dev = pd.read_csv(r'C:\Users\aaron\.cache\kagglehub\datasets\datamunge\sign-language-mnist\versions\1\sign_mnist_train\sign_mnist_train.csv').sample(frac=1, random_state=28)
X_eval, y_eval = data_eval.iloc[:, 1:], data_eval.iloc[:, 0]
X_dev, y_dev = data_dev.iloc[:, 1:], data_dev.iloc[:, 0]
X_dev.head(3)

In [ ]:
y_dev.head(3)

## Exploración

In [ ]:
# funcion auxiliar
def graficar_fila(row, ax, label=None):
    n = int(np.sqrt(len(row)))
    ax.imshow(row.values.reshape(n, n), cmap='gray')
    if label is not None:
        ax.set_title(label, fontsize='small')
    ax.axis('off')

leters = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y'}

fig, axs = plt.subplots(nrows=5, ncols=5, figsize=(6, 6))
for i, ax in enumerate(axs.flat):
    X_dev_i = X_dev[y_dev == i]
    if X_dev_i.shape[0] > 0:
        graficar_fila(X_dev_i.iloc[0], ax=ax, label=leters[i])
    else:
        ax.axis('off')
fig.suptitle('Ejemplos de cada seña')
fig.tight_layout()

In [ ]:
# proporcion de cada letra en el dataset de desarrollo
count = y_dev.value_counts().sort_index()
plt.figure(figsize=(5, 3))
sns.barplot(x=[leters.get(i, '?') for i in count.index], y=count.values)
plt.show()

Los datos estan balanceados, no hay necesidad de hacer stratified sampling.

In [ ]:
if X_dev.shape[0] == X_dev.value_counts().shape[0]: # no hay duplicados en el dataset de desarrollo
    print('No hay duplicados en el dataset de desarrollo')
else:
    print('Hay duplicados en el dataset de desarrollo, limpiar el dataset para evitar filtacion de datos')

## Un modelo simple
Arbol de profundidad maxima 10

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X_dev_train, X_dev_test, y_dev_train, y_dev_test = train_test_split(X_dev, y_dev, test_size=0.3, random_state=28)
tree = DecisionTreeClassifier(max_depth=10, random_state=28)
tree.fit(X_dev_train, y_dev_train)
y_dev_pred = tree.predict(X_dev_test)

print('Accuracy con test: ', accuracy_score(y_dev_test, y_dev_pred))

Veamos donde caen las features (pixeles) mas importantes para este modelo, ello veamos como se distribuyen. Esto nos dara un cota inferior razonable.

In [ ]:
plt.figure(figsize=(5, 3))
sns.boxplot(tree.feature_importances_)
plt.title('Importancia de los pixels')

In [ ]:
pixeles_mas_importantes = X_dev.columns[tree.feature_importances_ > 0.015] # cota elegida a ojo

fig, axs = plt.subplots(nrows=5, ncols=5, figsize=(6, 6))
for i, ax in enumerate(axs.flat):
    X_dev_i = X_dev[y_dev == i]
    if X_dev_i.shape[0] > 0:
        row = X_dev_i.iloc[0]
        row[pixeles_mas_importantes] = 0 # en negro
        graficar_fila(row, ax=ax, label=leters[i])
    else:
        ax.axis('off')
fig.suptitle('Ejemplos de cada seña')
fig.tight_layout()

Se observa que la mayoría de ellos se ubican sobre los dedos y contornos de la mano, lo que indica que el modelo está aprendiendo patrones coherentes con las diferencias morfológicas entre las clases. En algunos casos, como en las letras "C" o "L", los píxeles relevantes están claramente alineados con la forma característica del gesto, lo que refuerza la interpretación de que el modelo está focalizando en información útil.

### Validacion cruzada con 5 folds

In [ ]:
scoring = {
    'accuracy': 'accuracy',
    'auprc': 'average_precision',
    'roc_auc': 'roc_auc_ovr'
}

scores = cross_validate(DecisionTreeClassifier(max_depth=10), X_dev_test, y_dev_test, cv=5, scoring=scoring, n_jobs=n_jobs)

print(scores.keys())

In [ ]:
# Calculating mean scores for each metric
accuracy_scores = scores['test_accuracy']
auprc_scores = scores['test_auprc']
roc_auc_scores = scores['test_roc_auc']

mean_accuracy = np.mean(accuracy_scores)
mean_auprc = np.mean(auprc_scores)
mean_roc_auc = np.mean(roc_auc_scores)

# Creating a DataFrame for scores
scores_df = pd.DataFrame({
    'Fold': range(1, len(accuracy_scores) + 1),
    'Accuracy': accuracy_scores,
    'AUPRC': auprc_scores,
    'ROC AUC OneVSRest': roc_auc_scores
})

# Adding the mean row
mean_row = pd.DataFrame({
    'Fold': ['Mean'],
    'Accuracy': [mean_accuracy],
    'AUPRC': [mean_auprc],
    'ROC AUC OneVSRest': [mean_roc_auc]
})
scores_df = pd.concat([scores_df, mean_row], ignore_index=True)

# Displaying the table
print(scores_df)

Los resultados muestran que el modelo tiene un desempeño bastante flojo: la accuracy ronda el 50% y el AUPRC también es bajo. Aunque el ROC AUC por clase está un poco mejor (~0.85). En general, parece que el modelo no distingue bien entre las clases.

## Comparacion de algoritmos

Realizamos RandomizedSearchCV con Arboles de decision, KNN y SVM. Usamos como metrica de performance accuracy ya que tenemo un dataset balanceado (por lo menos los de desarrollo). Vale la pena aclarar que accuracy es de las peor metricas para evaluar un modelo, pero no existe un consenso una metrica integral para el clasificacion multiclase, además es la metrica que usa kaggle.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

tree_param_grid = {
    'max_depth': range(1,90), 'criterion': ['gini', 'entropy'],
    'random_state':[28]
}

knn_param_grid = {
    'n_neighbors': range(1,50)
}

svm_param_grid = {
    'C': np.logspace(-4, 2, 20),
    'random_state':[28]
}

rscv_tree = RandomizedSearchCV(DecisionTreeClassifier(), tree_param_grid, n_iter=10, cv=5, scoring='accuracy', n_jobs=n_jobs, random_state=28)
rscv_knn = RandomizedSearchCV(KNeighborsClassifier(), knn_param_grid, n_iter=10, cv=5, scoring='accuracy', n_jobs=n_jobs, random_state=28)
rscv_svm = RandomizedSearchCV(SVC(), svm_param_grid, n_iter=10, cv=5, scoring='accuracy', n_jobs=n_jobs, random_state=28)

rscv_tree.fit(X_dev_train, y_dev_train)
print('Mejores hiperparametros para arbol de decision: ', rscv_tree.best_params_)
print('         Accuracy: ', rscv_tree.best_score_)
print()

rscv_knn.fit(X_dev_train, y_dev_train)
print('Mejores hiperparametros para KNN: ', rscv_knn.best_params_)
print('         Accuracy: ', rscv_knn.best_score_)
print()

rscv_svm.fit(X_dev_train, y_dev_train)
print('Mejores hiperparametros para SVM: ', rscv_svm.best_params_)
print('         Accuracy: ', rscv_svm.best_score_)

Veamos que tal LDA y Naive Bayes

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn import naive_bayes

scores_LDA = cross_val_score(LinearDiscriminantAnalysis(), X_dev_train, y_dev_train, cv=4, n_jobs=n_jobs, scoring='accuracy')
print('Cross validation scores for LDA:', scores_LDA)
print('         Mean:', scores_LDA.mean())
print()

for func in [naive_bayes.GaussianNB, naive_bayes.MultinomialNB]:
    scores = cross_val_score(func(), X_dev_train, y_dev_train, cv=4, n_jobs=n_jobs, scoring='accuracy')
    print(f'Cross validation scores for {func.__name__}:', scores)
    print('            Mean:', scores.mean())
    print()

scores_RF = cross_val_score(RandomForestClassifier(random_state=28), X_dev_train, y_dev_train, cv=4, n_jobs=n_jobs, scoring='accuracy')
print('Cross validation scores for RandomForest:', scores_RF)
print('         Mean:', scores_RF.mean())
print()


## Sesgo y varianza

Estudiaremos mas a fondo el mejor arbol, Random Forest, SVM y Linear discriminant analysis (LDA)

### Curvas de complejidad

In [ ]:
def curva(model, X_train, X_test, y_train, y_test, param :str, steps):
    sub_train_error = []
    test_error = []
    for i in steps:
        model2 = model(**{param: i, 'random_state': 28})
        model2.fit(X_train, y_train)
        sub_train_error.append(1 - accuracy_score(y_train, model2.predict(X_train)))
        test_error.append(1 - accuracy_score(y_test, model2.predict(X_test)))
    return sub_train_error, test_error

In [ ]:
plt.figure(figsize=(12, 4))

cant = 10
max_complex = 50
steps = np.arange(1, max_complex, max_complex//cant, dtype=int)
y_1, y_2 = curva(DecisionTreeClassifier, X_dev_train, X_dev_test, y_dev_train, y_dev_test, 'max_depth', steps)

plt.subplot(1,3,1)
plt.plot(steps, y_1, label="Train error")
plt.plot(steps, y_2, label="Train error")
plt.title("Arbol")
plt.xlabel("max_depth")
plt.ylabel("1 - accuracy")
plt.ylim((0,1))
plt.legend()

y_1, y_2 = curva(RandomForestClassifier, X_dev_train, X_dev_test, y_dev_train, y_dev_test, 'n_estimators', steps)

plt.subplot(1,3,2)
plt.plot(steps, y_1, label="Train error")
plt.plot(steps, y_2, label="Test error")
plt.title("RandomForestClassifier")
plt.xlabel("n_estimators")
plt.ylabel("1 - accuracy")
plt.ylim((0,1))
plt.legend()

steps = np.linspace(1e-4, 120, cant)
y_1, y_2 = curva(SVC, X_dev_train, X_dev_test, y_dev_train, y_dev_test, 'C', steps)

plt.subplot(1,3,3)
plt.plot(steps, y_1, label="Train error")
plt.plot(steps, y_2, label="Test error")
plt.title("SVM")
plt.xlabel("C")
plt.ylabel("1 - accuracy")
plt.ylim((0,1))
plt.legend()

plt.suptitle('Curvas de complejidad')
plt.tight_layout()
plt.show()

Estas curvas de complejidad muestran cómo varía el error (medido como \(1 - \text{accuracy}\)) en el conjunto de entrenamiento y test a medida que se incrementa la complejidad del modelo, según un hiperparámetro característico:

- **Árbol de Decisión:** Se observa una clara mejora en el desempeño al aumentar `max_depth` hasta cierto punto, donde ambos errores convergen, esto sugiere que el modelo ha alcanzado un punto de sobreajuste.
- **Random Forest:** A medida que se incrementa el número de árboles (`n_estimators`), el error disminuye rápidamente, estabilizándose con un rendimiento sólido tanto en entrenamiento como en test.
- **SVM:** El parámetro `C` controla la penalización por errores de clasificación. Para valores bajos, el modelo subajusta fuertemente (altos errores). A partir de \(C $\sim$ 0.2\), el modelo logra ajustar correctamente, alcanzando un mínimo error de generalización.

Estas curvas no solo permiten comparar modelos, sino también entender el comportamiento de cada uno frente a la complejidad y cómo responden al ajuste de hiperparámetros clave.

### Curvas de aprendizaje

In [ ]:
from copy import deepcopy

def curva_de_aprendizaje(model, X_train, X_test, y_train, y_test):
    sub_train_error = []
    test_error = []
    for i in np.linspace(800, len(X_dev_train), 8, dtype=int):
        X_train_i = X_train.iloc[:i,:]
        y_train_i = y_train.iloc[:i]
        X_test_i  =  X_test.iloc[:i,:]
        y_test_i  =  y_test.iloc[:i]
        model2 = deepcopy(model)
        model2.fit(X_train_i, y_train_i)
        sub_train_error.append(1 - accuracy_score(y_train_i, model2.predict(X_train_i)))
        test_error.append(1 - accuracy_score(y_test_i, model2.predict(X_test_i)))
    return sub_train_error, test_error

In [ ]:
x = np.linspace(800, len(X_dev_train), 5)
plt.figure(figsize=(16, 4))

y_1, y_2 = curva_de_aprendizaje(DecisionTreeClassifier(**rscv_tree.best_params_), X_dev_train, X_dev_test, y_dev_train, y_dev_test)

plt.subplot(1,4,1)
plt.plot(x, y_1, label="Train error")
plt.plot(x, y_2, label="Test error")
plt.title("Arbol")
plt.xlabel("Tamanio train")
plt.ylabel("1 - accuracy")
plt.ylim((0,1))
plt.legend()

y_1, y_2 = curva_de_aprendizaje(RandomForestClassifier(random_state=28), X_dev_train, X_dev_test, y_dev_train, y_dev_test)

plt.subplot(1,4,2)
plt.plot(x, y_1, label="Train error")
plt.plot(x, y_2, label="Test error")
plt.title("RandomForestClassifier")
plt.xlabel("Tamanio train")
plt.ylabel("1 - accuracy")
plt.ylim((0,1))
plt.legend()

y_1, y_2 = curva_de_aprendizaje(SVC(**rscv_svm.best_params_), X_dev_train, X_dev_test, y_dev_train, y_dev_test)

plt.subplot(1,4,3)
plt.plot(x, y_1, label="Train error")
plt.plot(x, y_2, label="Test error")
plt.title("SVM")
plt.xlabel("Tamanio train")
plt.ylabel("1 - accuracy")
plt.ylim((0,1))
plt.legend()

y_1, y_2 = curva_de_aprendizaje(LinearDiscriminantAnalysis(), X_dev_train, X_dev_test, y_dev_train, y_dev_test)

plt.subplot(1,4,4)
plt.plot(x, y_1, label="Train error")
plt.plot(x, y_2, label="Test error")
plt.title("LDA")
plt.xlabel("Tamanio train")
plt.ylabel("1 - accuracy")
plt.ylim((0,1))
plt.legend()

plt.suptitle('Curvas de aprendizaje')
plt.tight_layout()
plt.show()

Las curvas de aprendizaje permiten analizar el comportamiento del error a medida que se incrementa el tamaño del conjunto de entrenamiento. Se presentan las curvas para cuatro modelos: Árbol de Decisión, Random Forest, SVM y LDA. En el eje \(x\) se muestra el tamaño del conjunto de entrenamiento, y en el eje \(y\), el error (\(1 - \text{accuracy}\)) para entrenamiento y test.

- **Árbol de Decisión:** El error de entrenamiento se mantiene cercano a cero, mientras que el error de test disminuye progresivamente con más datos. Esto sugiere un modelo con baja varianza pero cierto sesgo.
- **Random Forest:** Se logra un excelente desempeño con mas de una cuarta parte de los datos. El error de test converge rápidamente a un valor muy bajo, indicando robustez y buena generalización.
- **SVM:** También muestra una rápida convergencia del error de test, confirmando su capacidad de generalizar bien, probablemente gracias a la separación que logra el kernel gaussiano.
- **LDA:** Presenta un comportamiento similar a SVM, aunque su rendimiento es levemente inferior. Su naturaleza lineal parece ser suficiente para capturar bien la estructura de los datos en este caso.

En todos los casos, la brecha entre los errores de entrenamiento y test se reduce con mayor cantidad de datos, lo cual es un buen indicio de que los modelos no están sobreajustando. Además, se evidencia que más datos contribuyen a una mejora en la generalización.

A medida que aumenta el tamaño del conjunto de entrenamiento, la diferencia entre el error de entrenamiento y el de test tiende a reducirse, lo cual sugiere que los modelos están logrando una buena capacidad de generalización. En general, se observa que disponer de más datos tiene un efecto positivo en el desempeño de los modelos sobre datos no vistos.


## Evaluacion de performance

Elijo SVM como modelo final, ya que es el que mejor desempeño tiene en el conjunto de test. Aun así, la diferencia con Random Forest no es significativa.

In [ ]:
best = SVC(**rscv_svm.best_params_)
best.fit(X_dev, y_dev)
y_pred = best.predict(X_eval)

print('Accuracy con los datos de evaluacion: ', accuracy_score(y_eval, y_pred))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

fig, ax = plt.subplots(figsize=(9, 9))
ConfusionMatrixDisplay.from_predictions(
    y_eval, y_pred, display_labels=[leters[i] for i in leters.keys()],
    ax=ax, cmap='Blues', colorbar=False
)

plt.title('Matriz de confusión')

plt.tight_layout()
plt.show()